# Data processing

Download, save, visualize and load dataset.



In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import time
from pathlib import Path

In [2]:
DATA_DIR = "../../data/"
resolution = "5.625deg"

## 1. geopotential_500, temperature_850

### The data

- Test: years 2017 and 2018
- Validation: year 2016
- Train: year 2015

In [3]:
from torch.utils.data import Dataset
from torchvision import transforms

In [4]:
def preprocess_data(DATA_DIR, train_years, val_years, test_years):
    
    time_slices = {'train': train_years, 'val': val_years, 'test': test_years}
    
    
    zpath = DATA_DIR + '5.625deg/geopotential_500/'
    tpath = DATA_DIR + '5.625deg/temperature_850/'
    
    z = xr.open_mfdataset(zpath+'/*.nc', combine='by_coords')['z'].assign_coords(level=1)
    t = xr.open_mfdataset(tpath+'/*.nc', combine='by_coords')['t'].assign_coords(level=1)

    ratio = len(z.coords['lon'])/len(z.coords['lat'])

    data = xr.concat([z, t], 'level').stack(v=('lat', 'lon')).transpose('time', 'v', 'level').drop('level')
    
    data_paths = []
    for set_name in ['train', 'val', 'test']:
    
        # Create directory
        out_path = DATA_DIR + set_name + "/"
        Path(out_path).mkdir(parents=True, exist_ok=True)
        data_paths.append(out_path)
        
        # Select relevant years
        dataset = data.sel(time=time_slices[set_name])

        # Compute mean and std
        mean = data.mean(('time', 'v')).compute()
        std = data.std('time').mean(('v')).compute()
        np.save(out_path + 'mean.npy', mean.values)
        np.save(out_path + 'std.npy', std.values)
    
        # Save individual arrays
        for i, array in enumerate(dataset):
            np.save(out_path + str(i) + '.npy', array.values)
    
    return tuple(data_paths)
    

In [ ]:
train_path, val_path, test_path = preprocess_data(DATA_DIR, 
                                                  train_years=slice('2015', '2015'), 
                                                  val_years=slice('2016', '2016'), 
                                                  test_years=slice('2017', '2018'))

In [466]:
class WeatherBenchDataset(Dataset):
    
    def __init__(self, data_path, delta_t):
        
        self.delta_t = delta_t
        
        self.mean = np.load(data_path + 'mean.npy')
        self.std = np.load(data_path + 'std.npy')
        
        '''self.transform = transforms.Compose([torch.Tensor(), 
                                              transforms.Normalize(mean=self.mean, std=self.std)])'''
        
        total_samples = len(os.listdir(data_path)) - 2
        datafiles = [data_path+str(id)+'.npy' for id in list(range(total_samples))]
        self.datafiles = datafiles[:-self.delta_t]
        
        self.n_samples = len(self.datafiles)
    
    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, idx):
        """ Returns sample and label corresponding to an index as torch.Tensor objects
            The return tensor shapes are (for the sample and the label): [n_vertex, n_features]
        """
        
        '''X = self.transform(np.load(self.datafiles[idx]))
        y = self.transform(np.load(self.datafiles[idx+delta_t]))'''
        
        X = torch.Tensor((np.load(self.datafiles[idx])-self.mean)/self.std)
        y = torch.Tensor((np.load(self.datafiles[idx+delta_t])-self.mean)/self.std)
        
        return X, y

### The graphs

Bandwidth = [dim1/2, dim2/2]  
sampling = 'SOFT'

In [ ]:
def compute_laplacian(nodes, ratio, laplacian_type):
    dim1, dim2 = equiangular_dimension_unpack(nodes, ratio)
    
    bw = [int(dim1/2), int(dim2/2)]

    G = SphereEquiangular(bandwidth=bw, sampling="SOFT")
    G.compute_laplacian(laplacian_type)
    laplacian = prepare_laplacian(G.L)
    
    return laplacian

In [9]:
from deepsphere.utils.laplacian_funcs import prepare_laplacian
from deepsphere.utils.samplings import equiangular_dimension_unpack

def get_equiangular_laplacians2(nodes, ratio, depth, laplacian_type, pool_size):
    """Get the equiangular laplacian list for a certain depth.
    Args:
        nodes (tuple): input signal size (lat x lon)
        depth (int): the depth of the UNet.
        laplacian_type ["combinatorial", "normalized"]: the type of the laplacian.
        pool_size: size of the pooling kernel
    Returns:
        laps (list): increasing list of laplacians
    """
    laps = []
    dim1, dim2 = equiangular_dimension_unpack(nodes, ratio)
    
    for i in range(depth):
        # Adjust dimensions with depth!
        bw1 = int(dim1/(2*pool_size**i))
        bw2 =int(dim2/(2*pool_size**i))
        bw = [bw1, bw2]
        
        G = SphereEquiangular(bandwidth=bw, sampling="SOFT")
        G.compute_laplacian(laplacian_type)
        laplacian = prepare_laplacian(G.L)
        laps.append(laplacian)
    return laps

### The models

https://github.com/ArcaniteSolutions/deepsphere

In [14]:
from torch import nn

from deepsphere.models.spherical_unet.encoder import SphericalChebBN2
from deepsphere.models.spherical_unet.utils import SphericalChebBNPool
from deepsphere.models.spherical_unet.decoder import SphericalChebBNPoolConcat, SphericalChebBNPoolCheb


from deepsphere.layers.chebyshev import SphericalChebConv
from deepsphere.utils.laplacian_funcs import get_equiangular_laplacians
from deepsphere.layers.samplings.equiangular_pool_unpool import Equiangular, reformat
from deepsphere.utils.samplings import equiangular_calculator
import torch.nn.functional as F

In [133]:
class EquiangularMaxPool2(nn.MaxPool1d):
    """EquiAngular Maxpooling module using MaxPool 1d from torch
    """

    def __init__(self, ratio, kernel_size, return_indices=False):
        """Initialization
        Args:
            ratio (float): ratio between latitude and longitude dimensions of the data
        """
        self.ratio = ratio
        super().__init__(kernel_size=kernel_size, return_indices=return_indices)

    def forward(self, x):
        """calls Maxpool1d and if desired, keeps indices of the pixels pooled to unpool them
        Args:
            input (:obj:`torch.tensor`): batch x pixels x features
        Returns:
            tuple(:obj:`torch.tensor`, list(int)): batch x pooled pixels x features and the indices of the pixels pooled
        """
        x, _ = equiangular_calculator(x, self.ratio)
        x = x.permute(0, 3, 1, 2)

        if self.return_indices:
            x, indices = F.max_pool2d(x, self.kernel_size, return_indices=self.return_indices)
        else:
            x = F.max_pool2d(x, self.kernel_size)
        x = reformat(x)

        if self.return_indices:
            output = x, indices
        else:
            output = x

        return output


class EquiangularMaxUnpool2(nn.MaxUnpool1d):
    """Equiangular Maxunpooling using the MaxUnpool1d of pytorch
    """

    def __init__(self, ratio, kernel_size):
        """Initialization
        Args:
            ratio (float): ratio between latitude and longitude dimensions of the data
        """
        self.ratio = ratio
        
        super().__init__(kernel_size=(kernel_size, kernel_size))

    def forward(self, x, indices):
        """calls MaxUnpool1d using the indices returned previously by EquiAngMaxPool
        Args:
            x (:obj:`torch.tensor`): batch x pixels x features
            indices (int): indices of pixels equiangular maxpooled previously
        Returns:
            :obj:`torch.tensor`: batch x unpooled pixels x features
        """
        x, _ = equiangular_calculator(x, self.ratio)
        x = x.permute(0, 3, 1, 2)
        x = F.max_unpool2d(x, indices, self.kernel_size)
        x = reformat(x)
        return x

In [163]:
class SphericalConvNet(nn.Module):
    """Spherical GCNN Autoencoder.
    """

    def __init__(self, nodes, ratio, depth, channels_in, channels_out, laplacian_type, kernel_size):
        """Initialization.
        Args:
            N (int): Number of pixels in the input image
            depth (int): The depth of the UNet, which is bounded by the N and the type of pooling
            kernel_size (int): chebychev polynomial degree
            ratio (float): Parameter for equiangular sampling -> width/height
        """
        super().__init__()
        
        self.kernel_size = kernel_size
        self.laplacian = compute_laplacian(nodes, ratio, laplacian_type)
        
        self.conv1 = SphericalChebConv(channels_in, 64, self.laplacian, self.kernel_size)
        self.conv2 = SphericalChebConv(64, 64, self.laplacian, self.kernel_size)
        self.conv3 = SphericalChebConv(64, 64, self.laplacian, self.kernel_size)
        self.conv4 = SphericalChebConv(64, 64, self.laplacian, self.kernel_size)
        self.conv5 = SphericalChebConv(64, channels_out, self.laplacian, self.kernel_size)

    def forward(self, x):
        """Forward Pass.
        Args:
            x (:obj:`torch.Tensor`): input to be forwarded.
        Returns:
            :obj:`torch.Tensor`: output
        """
        
        x = F.elu(self.conv1(x))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        x = F.elu(self.conv4(x))
        x = F.elu(self.conv5(x))
        
        if not self.training:
            x = self.softmax(x)
        return x
        
        return x

In [163]:
class Encoder2(nn.Module):
    """
    SphericalChebConv has "same" padding
    
    """
    def __init__(self, channels_in, kernel_size, ratio, laplacians, pool_size, pooling="max"):
    
        super().__init__()

        self.kernel_size = kernel_size

        self.conv1 = SphericalChebConv(channels_in, 64, laplacians[0], self.kernel_size)
        self.conv2 = SphericalChebConv(64, 128, laplacians[1], self.kernel_size)
        self.conv3 = SphericalChebConv(128, 128, laplacians[2], self.kernel_size)
        self.conv4 = SphericalChebConv(128, 128, laplacians[3], self.kernel_size)

        self.pool = EquiangularMaxPool2(ratio, pool_size, return_indices=True)
        

    def forward(self, x):
        """Forward Pass.
        Args:
            x (:obj:`torch.Tensor`): input [batch x vertices x channels/features]
        Returns:
            x_enc* :obj: `torch.Tensor`: output [batch x vertices x channels/features]
        """
        
        x_enc1 = F.relu(self.conv1(x))
        
        x_enc2, idx2 = self.pool(x_enc1)
        x_enc2 = F.relu(self.conv2(x_enc2))
        
        x_enc3, idx3 = self.pool(x_enc2)
        x_enc3 = F.relu(self.conv3(x_enc3))
        
        x_enc4, idx4 = self.pool(x_enc3)
        x_enc4 = self.conv4(x_enc4)

        return x_enc2, idx2, x_enc3, idx3, x_enc4, idx4

class Decoder2(nn.Module):
    """The decoder of the Spherical UNet.
    """
    
    def __init__(self, channels_out, kernel_size, ratio, laplacians, pool_size, pooling="max"):
        """Initialization.
        Args:
            unpooling (:obj:`torch.nn.Module`): The unpooling object.
            laps (list): List of laplacians.
        """
        super().__init__()
        self.kernel_size = kernel_size
        
        self.unpool = EquiangularMaxUnpool2(ratio, pool_size)

        self.deconv3 = SphericalChebConv(128, 128, laplacians[2], self.kernel_size)
        self.deconv2 = SphericalChebConv(128, 64, laplacians[1], self.kernel_size)
        self.deconv1 = SphericalChebConv(64, channels_out, laplacians[0], self.kernel_size)
        
        self.conv3 = SphericalChebConv(128+128, 128, laplacians[2], self.kernel_size)
        self.conv2 = SphericalChebConv(128+64, 64, laplacians[1], self.kernel_size)

        # Switch from Logits to Probabilities if evaluating model
        self.softmax = nn.Softmax(dim=2)

    def forward(self, x_enc2, idx2, x_enc3, idx3, x_enc4, idx4):
        """Forward Pass.
        Args:
            x_enc* (:obj:`torch.Tensor`): input tensors.
        Returns:
            :obj:`torch.Tensor`: output after forward pass.
        """
        
        x = self.unpool(x_enc4, idx4)
        x = F.relu(self.deconv3(x))
        x = torch.cat((x, x_enc3), dim=2)
        x = F.relu(self.conv3(x))
        
        x = self.unpool(x, idx3)
        x = F.relu(self.deconv2(x))
        x = torch.cat((x, x_enc2), dim=2)
        x = F.relu(self.conv2(x))
        
        x = self.unpool(x, idx2)
        x = F.relu(self.deconv1(x))
        
        if not self.training:
            x = self.softmax(x)
        return x
    
class SphericalUNet2(nn.Module):
    """Spherical GCNN Autoencoder.
    """

    def __init__(self, nodes, ratio, depth, channels_in, channels_out, laplacian_type, kernel_size, pooling_size, 
                 pooling="max"):
        """Initialization.
        Args:
            N (int): Number of pixels in the input image
            depth (int): The depth of the UNet, which is bounded by the N and the type of pooling
            kernel_size (int): chebychev polynomial degree
            ratio (float): Parameter for equiangular sampling -> width/height
        """
        super().__init__()
        self.kernel_size = kernel_size
        
        self.pooling_class = Equiangular(mode=pooling)
        self.laplacians = get_equiangular_laplacians2(nodes, ratio, depth, laplacian_type, pooling_size)

        self.encoder = Encoder2(channels_in, self.kernel_size, ratio, self.laplacians, pooling_size, pooling)
        self.decoder = Decoder2(channels_out, self.kernel_size, ratio, self.laplacians, pooling_size, pooling)

    def forward(self, x):
        """Forward Pass.
        Args:
            x (:obj:`torch.Tensor`): input to be forwarded.
        Returns:
            :obj:`torch.Tensor`: output
        """
        x_encoder = self.encoder(x)
        output = self.decoder(*x_encoder)
        return output

### Train 

In [18]:
import torch

from torch.utils.data import DataLoader
from torch import nn, optim 

In [486]:
def train_model(model, lr, device, train_generator, val_generator, patience):
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    min_val_loss = float("inf")
    epoch_no_improve = 0
    
    
    for epoch in range(n_epochs):
        val_loss = 0
        loss = 0
        
        model.train()
        
        for batch, labels in train_generator:
            # Transfer to GPU
            batch, labels = batch.to(device), labels.to(device)
            
            
            # Model
            output = model(batch)

            loss = criterion(output, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            
        model.eval()
        with torch.set_grad_enabled(False):
            for batch, labels in val_generator:
                # Transfer to GPU
                batch, labels = batch.to(device), labels.to(device)
                
                output = model(batch)

                val_loss = val_loss + criterion(output, labels).item()/len(train_generator)
                
        # Print stuff
        print('Epoch: {e:3d}/{n_e:3d}  - loss: {l:.3f}  - val_loss: {v_l:.3f}'.format(e=epoch, n_e=n_epochs, 
                                                                                      l=loss, v_l=val_loss))
                
        # Check for early stopping
        if val_loss < min_val_loss:
            epoch_no_improve = 0
            min_val_loss = val_loss
        else:
            epoch_no_improve += 1

        if epoch_no_improve == patience:
            print('Epoch {e:3d}: early stopping'.format(e=epoch))
            torch.save(model.state_dict(), 'model.pt')
            break
            
    torch.save(model.state_dict(), 'model.pt')

In [487]:
def init_device(model, ids=None):
    """Initialize device based on cpu/gpu and number of gpu
    Args:
        device (str): cpu or gpu
        ids (list of int or str): list of gpus that should be used
        unet (torch.Module): the model to place on the device(s)
    Raises:
        Exception: There is an error in configuring the cpu or gpu
    Returns:
        torch.Module, torch.device: the model placed on device, the device
    """
    
    if torch.cuda.is_available():
        if ids is None:
            device = torch.device("cuda")
            model = model.to(device)
            unet = nn.DataParallel(unet)
        elif len(ids) == 1:
            device = torch.device("cuda:{}".format(ids[0]))
            model = model.to(device)
        else:
            device = torch.device("cuda:{}".format(ids[0]))
            model = model.to(device)
            model = nn.DataParallel(model, device_ids=[int(i) for i in ids])
        #cudnn.benchmark = True
    else:
        device = torch.device("cpu")
        model = model.to(device)

    return model, device

In [ ]:
delta_t = 3*24
ratio = 64/32

batch_size = 68
learning_rate = 1e-4
n_epochs = 20

# Data
training_set = WeatherBenchDataset(train_path, delta_t)
validation_set = WeatherBenchDataset(val_path, delta_t)
dataloader_train = DataLoader(training_set, batch_size=batch_size, shuffle=False, num_workers=10)
dataloader_validation = DataLoader(validation_set, batch_size=batch_size, shuffle=False, num_workers=10)

In [ ]:
# Model - CONVNET
convnet = SphericalConvNet(nodes=N, ratio=ratio, depth=4, channels_in=2, channels_out=2, "combinatorial", 
                           kernel_size=5)

convnet, device = init_device(model=unet, ids=[0])

In [ ]:
# Model - UNET
unet = SphericalUNet2(nodes=N, ratio=ratio, depth=4, channels_in=2, channels_out=2, laplacian_type='combinatorial', 
                     kernel_size=5, pooling_size=2, pooling='max')

unet, device = init_device(model=unet, ids=[0])

In [497]:
train_model(convnet, learning_rate, device, dataloader_train, dataloader_validation, patience=2)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff55d02a3b0>
Traceback (most recent call last):
  File "/home/illorens/miniconda3/envs/weather_modelling/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/home/illorens/miniconda3/envs/weather_modelling/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/home/illorens/miniconda3/envs/weather_modelling/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff55d02a3b0>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff55d02a3b0>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/illorens/miniconda3/envs/weather_modellin

IndexError: Caught IndexError in DataLoader worker process 3.
Original Traceback (most recent call last):
  File "/home/illorens/miniconda3/envs/weather_modelling/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/illorens/miniconda3/envs/weather_modelling/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/illorens/miniconda3/envs/weather_modelling/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-466-4804c5ca6a74>", line 31, in __getitem__
    y = torch.Tensor((np.load(self.datafiles[idx+delta_t])-self.mean)/self.std)
IndexError: list index out of range


In [214]:
def MSE(yhat, y):
    return torch.mean((yhat-y)**2)

In [499]:
testing_set = WeatherBenchDataset(test_path, delta_t)
dataloader_test = DataLoader(testing_set, batch_size=32, shuffle=False)

'''trained_unet = SphericalUnet(N=N, depth=3, channels_in=2, channels_out=2, laplacian_type='combinatorial', 
                     kernel_size=5, ratio=ratio, pooling='max')
trained_unet.load_state_dict(torch.load('model.pt'))
trained_unet.eval()'''

mse = torch.tensor(0, requires_grad=False).to(device)
evaluation = nn.MSELoss()

unet.eval()
with torch.set_grad_enabled(False):
    for batch_idx, (batch, labels) in enumerate(dataloader_test):
        batch, labels = batch, labels
        batch, labels = batch.to(device), labels.to(device)

        output = unet(batch)

        mse = mse + MSE(output, labels)

RMSE = torch.sqrt(mse/len(dataloader_test)).item()

0/168
1/168
2/168
3/168
4/168
5/168
6/168
7/168
8/168
9/168
10/168
11/168
12/168
13/168
14/168
15/168
16/168
17/168
18/168
19/168
20/168
21/168
22/168
23/168
24/168
25/168
26/168
27/168
28/168
29/168
30/168
31/168
32/168
33/168
34/168
35/168
36/168
37/168
38/168
39/168
40/168
41/168
42/168
43/168
44/168
45/168
46/168
47/168
48/168
49/168
50/168
51/168
52/168
53/168
54/168
55/168
56/168
57/168
58/168
59/168
60/168
61/168
62/168
63/168
64/168
65/168
66/168
67/168
68/168
69/168
70/168
71/168
72/168
73/168
74/168
75/168
76/168
77/168
78/168
79/168
80/168
81/168
82/168
83/168
84/168
85/168
86/168
87/168
88/168
89/168
90/168
91/168
92/168
93/168
94/168
95/168
96/168
97/168
98/168
99/168
100/168
101/168
102/168
103/168
104/168
105/168
106/168
107/168
108/168
109/168
110/168
111/168
112/168
113/168
114/168
115/168
116/168
117/168
118/168
119/168
120/168
121/168
122/168
123/168
124/168
125/168
126/168
127/168
128/168
129/168
130/168
131/168
132/168
133/168
134/168
135/168
136/168
137/168
138/16

IndexError: list index out of range